# Overwrite Content with CSV

The script utilizes pyodbc and pandas dataframes to read in data from a SQL Database.  Export panda dataframe into CSV
file, overwrite service in ArcGIS Online.

In [1]:
# Import libraries
import pyodbc
import pandas as pd
from arcgis.gis import GIS
from arcgis import features
from getpass import getpass 

In [4]:
# Connect using pyodbc into SQL Server
sql_conn = pyodbc.connect(
    'DRIVER={ODBC Driver 13 for SQL Server}; SERVER=abrown10; DATABASE=Enterprise; Trusted_Connection=yes')

# Establish csv output
csv_output = r'D:\Scratch\PhillyData1.csv'

# Construct query and execute
query = "SELECT point_y, point_x, dc_dist, dispatch_1, dispatch_t from SDE.PHILLYCRIMESUBSET"

# Read in query into Panda dataframe
df = pd.read_sql(query, sql_conn)

# Establish Columns and show header
df.columns = ['Latitude', 'Longitude', 'dc_dist', 'dispatch_1', 'dispatch_t']
df.head()

,Latitude,Longitude,dc_dist,dispatch_1,dispatch_t
0,40.062614,-75.043076,07,2015-04-10,02:24:00
1,40.063133,-75.042185,08,2015-09-18,11:50:00
2,40.063985,-75.041810,08,2015-08-19,14:38:00
3,40.063133,-75.042185,08,2015-04-30,22:53:00
4,40.063133,-75.042185,08,2015-08-07,10:50:00


In [5]:
# Export to a csv file
df.to_csv(csv_output, sep=',', index=False)

In [29]:
# Read the initial csv
csv1 = csv_output
cities_df_1 = pd.read_csv(csv1)
cities_df_1.head()

,Latitude,Longitude,dc_dist,dispatch_1,dispatch_t
0,40.062614,-75.043076,7,2015-04-10,02:24:00
1,40.063133,-75.042185,8,2015-09-18,11:50:00
2,40.063985,-75.041810,8,2015-08-19,14:38:00
3,40.063133,-75.042185,8,2015-04-30,22:53:00
4,40.063133,-75.042185,8,2015-08-07,10:50:00


In [7]:
# Print the number of records & columns
cities_df_1.shape

(55, 5)

In [8]:
# Connect to the GIS
gis = GIS("https://www.arcgis.com",'abrown_citygov')

Enter password: ········


In [31]:
# Add the csv as an item in ArcGIS Online/Enterprise
item_prop = {'title':'Philly Crime GeoDev Test'}
csv_item = gis.content.add(item_properties=item_prop, data=csv1)

In [32]:
# Publish the csv item as a feature service
new_item = csv_item.id
feature_layer_item = gis.content.get(new_item)
feature_layer_item.publish()
feature_layer_item

<Item title:"Philly Crime GeoDev Test" type:CSV owner:abrown_citygov>

In [33]:
# Search by title
search = gis.content.search(query="title: Philly Crime GeoDev Test", item_type="Feature Service")
search

[<Item title:"Philly Crime GeoDev Test" type:Feature Layer Collection owner:abrown_citygov>]

In [34]:
# From search list, return the index for service one is interseted in connecting to
item = search[0]
itemID = item.id
itemID

'4f827de3f54e4d22b57e15447343d793'

In [35]:
# Establish connection to Feature Layer item
feature_layer_new = gis.content.get(itemID)

In [39]:
# Create a new map, zoom to Rhawnhurst, PA (Neighborhood of Philadelphia)
map1 = gis.map("Rhawnhurst, PA")
map1

MapView(layout=Layout(height='400px', width='100%'))

In [40]:
# Add new service into the map
map1.add_layer(feature_layer_new)